In [1]:
import pandas as pd
import geopandas as gpd
import psycopg2 as psql
import numpy as np
from sklearn.preprocessing import StandardScaler

In [23]:
def fetch_table(query, connect):
    '''
    Fetch table from database
    Input: str, str / Output: Dataframe
    '''
    cur = connect.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    cur.close()
    return df

In [24]:
db_name = 'SubwayDWT'
conn = psql.connect(host = 'localhost', port = 5432, user = 'postgres', password = 'user', database = db_name)
cur = conn.cursor()

In [7]:
conn.close()

In [25]:
query = 'SELECT * FROM subwayinfo_x'
raw_x = fetch_table(query, conn)

In [26]:
col_rename = {0: 'id', 1: 'geom', 2: 'index', 3: 'name', 4: 'transfer', 5: 'outer', 6: 'terminal', 7: 'cbd',
              8: 'tfa_r', 9: 'tfa_c', 10: 'tfa_w', 11: 'univ', 12: 'pop', 13: 'job', 14: 'bus',
              15: 'buld_sum', 16: 'rest', 17: 'green', 18: 'tfa_n', 19:'tfa_f'}
raw_x.rename(columns = col_rename, inplace = True)

In [27]:
data_x = raw_x.drop(columns = ['index', 'geom'])

In [28]:
#Get land use mix entropy
for k, v in data_x.iterrows():
    p1, p2, p3, p4, p5 = float(v['tfa_r'])/float(v['buld_sum']), float(v['tfa_c'])/float(v['buld_sum']), float(v['tfa_w'])/float(v['buld_sum']), float(v['tfa_n'])/float(v['buld_sum']), float(v['tfa_f'])/float(v['buld_sum'])
    data_x.at[k,'lumix'] = -1 * (np.log(p1)*p1 + np.log(p2)*p2 + np.log(p3)*p3 + np.log(p4)*p4 + np.log(p5)*p5)

In [29]:
data_x

,id,name,transfer,outer,terminal,cbd,tfa_r,tfa_c,tfa_w,univ,pop,job,bus,buld_sum,rest,green,tfa_n,tfa_f,lumix
0,8,문정,0,1,0,0,595029.3087675537361,10703.90254576616,696574.8013299960329,0,25099,81939,16,2621679.3758065227217,235.0,5572.8570581794,238224.3392143694227,1081147.02394883737,1.294415
1,6,가락시장,1,1,0,0,944586.7675563124951,736955.778443494229,387790.658052818856,0,19987,76521,19,2205514.2283421135152,402.0,14522.6853313655,136180.0242894879351,1,1.207038
2,81,길음,0,1,0,0,1299724.17983291052402,17039.07768617840798,7735.1507916201982,0,33785,10339,34,1424059.86603225783469,186.0,68232.088858311,98720.25074981660849,841.2069717320960,0.354080
3,111,용마산,0,0,0,0,873676.48069355470844,6194.28918489601728,12197.0118465917,0,29562,7478,13,1038716.12204231828157,224.0,208236.288455684,142991.92133711284685,3656.418980163009,0.521137
4,240,서울대입구,0,0,0,1,803222.53320797889559,32867.58991102873956,390383.7483746716650,0,41404,20820,23,1801035.50159464026157,916.0,6350.25020554751,572014.66972328688142,2546.96037767408,1.138154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,298,서원,0,0,0,0,999273.96541704114468,6583.094782896552,14637.278241295510,0,36212,9058,21,1198390.82394226408065,251.0,109236.090215756,177895.48550103087397,1,0.517085
298,292,금천구청,0,1,0,0,1214539.4331385164865,810.838854711394,109804.034506082573,0,19339,8593,18,1460942.7457965355535,141.0,27069.7172839947,98987.6225625781970,36800.8167346469030,0.627367
299,288,가산디지털단지,1,0,0,0,178777.55364002690766,77717.816359611919,374239.2348634699,0,11730,168764,33,3581268.48521120232192,745.0,None,246548.5739774172333,2703985.30637067636196,0.865149
300,293,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,295862.5547484429950,0,18101,29880,19,1322442.82543809803817,311.0,None,268628.39176299161746,189208.9416908443121,1.437046


In [30]:
#Standardization
scaler = StandardScaler()
norm_cols = ['tfa_r', 'tfa_c', 'tfa_w', 'tfa_n', 'tfa_f', 'pop', 'job', 'bus', 'buld_sum', 'rest', 'green']
for col in norm_cols:
    col2d = np.array(data_x[col]).reshape(-1, 1)
    data_x[col] = scaler.fit_transform(col2d)

In [31]:
data_x.to_csv('../Data/train_x.csv')

In [44]:
data_y = pd.read_csv('../Data/train_y.csv', index_col = 0)

In [46]:
train = data_x.merge(data_y, on = ['id', 'name'])

In [47]:
train.head(3)

,id,name,transfer,outer,terminal,cbd,tfa_r,tfa_c,tfa_w,univ,...,tfa_f,lumix,index,weekend,hacha,max_ride,min_ride,pattern,cA,cD
0,8,문정,0,1,0,0,-0.481898,-0.456531,1.807274,0,...,5.055451,1.294415,2,0,0,5700.4,53.8,"[0.0076860411575107165, 0.052385506322388704, ...",[0.02667335 0.0564326 0.33003991 0.15867922 0...,[-0.02737272 0.02047572 -0.02081252 -0.008290...
1,8,문정,0,1,0,0,-0.481898,-0.456531,1.807274,0,...,5.055451,1.294415,2,0,1,8018.8,84.8,"[0.008923619863876985, 0.06289387446433072, 0....",[ 3.17012707e-02 -1.06932741e-03 1.07434269e+...,[-3.30498963e-02 -2.31676900e-01 4.78929426e-...
2,8,문정,0,1,0,0,-0.481898,-0.456531,1.807274,0,...,5.055451,1.294415,2,1,0,820.0,1.5,"[0.07208307880268784, 0.14783139890042762, 0.2...",[0.12872194 0.15624499 0.52097392 0.81215442 1...,[-0.04638618 -0.05079307 0.05033163 -0.046363...


In [48]:
weekday_seungcha = train[(train['hacha'] == 0) & (train['weekend'] == 0)]
weekday_hacha = train[(train['hacha'] == 1) & (train['weekend'] == 0)]
weekend_seungcha = train[(train['hacha'] == 0) & (train['weekend'] == 1)]
weekend_hacha = train[(train['hacha'] == 1) & (train['weekend'] == 1)]

weekday_seungcha.to_csv('../Data/train_wd_sc.csv')
weekday_hacha.to_csv('../Data/train_wd_hc.csv')
weekend_seungcha.to_csv('../Data/train_we_sc.csv')
weekend_hacha.to_csv('../Data/train_we_hc.csv')